# Fluxo Novo

Ordem das células

- Importando base de dados
- Definindo lojas que estarão no relatório
- Definindo dia do relatório
- Definindo emails que receberão o relatório
- Rodando o código para a criação do relatório
- Preparando o html
- Rodando Mock do disparador de email e disparando email

## importando base de dados

In [105]:
from commom.database.queries.query_lojas import QUERY_LOJAS
from commom.database.queries.query_vendas_nf_novo import QUERY_VENDAS_NF_NOVO
from commom.database.queries.query_metas import QUERY_METAS
from commom.database.queries.query_parcelas import QUERY_PARCELAS
from commom.database.queries.query_vendas_nf import QUERY_VENDAS_NF
from commom.database.queries.query_vendedores import QUERY_VENDEDORES
from commom.kpi_data.kpi_data_manager import DataHandler, KpiDataManager
from typing import Tuple
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
from innovation_messenger import Messenger
import os
from innovation_messenger import IEmailProperties

manager = KpiDataManager()
handler = DataHandler()

# Service
## -> inicio
###     -> fetch data from local pickle
# df_novo = handler.read_from_local_pickle('fato_vendas.pkl')
df_lojas = handler.read_from_local_pickle('df_lojas.pkl')
df_vendedores = handler.read_from_local_pickle('df_vendedores.pkl')
df_vendas = handler.read_from_local_pickle('df_vendas.pkl')
df_parcelas = handler.read_from_local_pickle('df_parcelas.pkl')
df_metas = handler.read_from_local_pickle('df_metas.pkl')

###     -> fetch data from bq
# df = handler.read_from_bigquery(QUERY_VENDAS_NF_NOVO)
# df_lojas = handler.read_from_bigquery(QUERY_LOJAS)
# df_vendedores = handler.read_from_bigquery(QUERY_VENDEDORES)
# df_vendas = handler.read_from_bigquery(QUERY_VENDAS_NF)
# df_parcelas = handler.read_from_bigquery(QUERY_PARCELAS)
# df_metas = handler.read_from_bigquery(QUERY_METAS).rename(
#         columns={"cpf_vendedor": "cpf_vendedor_inteiro", "loja": "distributorId"}
#     )

# ----------------------------------------------------------------------
# ----------------------------------------------------------------------
# df_novo['createdAt'] = pd.to_datetime(df_novo["createdAt"])
# maskDfNovoDate = df_novo["createdAt"].dt.year >= 2024
# df_fato_vendas = df_novo[maskDfNovoDate]

### definindo loja id e data

- lista de lojas disponíveis nos dois dataframes
  ['10H1', '10I9', '10J3', '1077', '10I1', '10I4', '10J9', '10G2', '1086', '10H5', '10G6', '10J4', '10J7', '10i6', '10I8', '0091', '10J0', '10J5', '10G8', '10I3', '10J1', '10J8', '10L9', '10L7', '10L8', '10L0', '10I5', '10J2', '10M1', '10M0', '10F9', '10C4', '10M2', '1010', '1012', '1017', '1018', '1021', '1022', '1025', '1027', '1028', '1036', '1043', '1044', '1045', '1049', '1052', '1054', '1055', '1065', '1068', '1071', '1089', '10C2', '10E2', '10E4', '10E5', '10F3', '10F8']
- index deve estar entre 0 e 60
- data deve ser entre 1/jan/2024 até 31/dez/2024
- formato da data datetime.datetime(2024, 10, 6, 0, 0)

In [ ]:
# debug
# fato_vendas_copy = df_fato_vendas.copy()
# vendas_copy = df_vendas.copy()

# PDV_TYPE_LIST = ['Propria']

# PDVTYPELIST = ["STORE", "PI - 360", "PI", "EP", "REFUND"]

# maskpdv = vendas_copy["tipo_transacao"].isin(PDVTYPELIST)

# vendas_copy = vendas_copy[maskpdv]
# vendas_copy.loc[:, "tipo_transacao"] = vendas_copy["tipo_transacao"].replace("PI - 360", "STORE")
# vendas_copy.loc[:, "tipo_transacao"] = vendas_copy["tipo_transacao"].replace("PI", "STORE")
# vendas_copy.loc[:, "tipo_transacao"] = vendas_copy["tipo_transacao"].replace("EP", "STORE")


# mask_pdv = fato_vendas_copy["type"].isin(PDV_TYPE_LIST)
# maksNull = fato_vendas_copy['distributorId'].isnull()

# fato_vendas_copy = fato_vendas_copy[mask_pdv & ~maksNull]
# fatovendasLojasIds = fato_vendas_copy['distributorId'].unique().tolist()
# vendasLojasIds = vendas_copy['distributorId'].unique()

# currentStores = []
# for store in fatovendasLojasIds:
#     if store in vendasLojasIds:
#         currentStores.append(store)

# print(currentStores)
# len(currentStores)

## Definindo lojas que estarão no relatório

In [2]:
# codigo para pegar loja e data aleatoria
import random
import datetime

currentStores = ['10H1', '10I9', '10J3', '1077', '10I1', '10I4', '10J9', '10G2', '1086', '10H5', '10G6', '10J4', '10J7', '10i6', '10I8', '0091', '10J0', '10J5', '10G8', '10I3', '10J1', '10J8', '10L9', '10L7', '10L8', '10L0', '10I5', '10J2', '10M1', '10M0', '10F9', '10C4', '10M2', '1010', '1012', '1017', '1018', '1021', '1022', '1025', '1027', '1028', '1036', '1043', '1044', '1045', '1049', '1052', '1054', '1055', '1065', '1068', '1071', '1089', '10C2', '10E2', '10E4', '10E5', '10F3', '10F8']

numero_aleatorio_loja = random.randint(0, 60)

lojaAtual = currentStores[numero_aleatorio_loja]
print('loja', lojaAtual)

loja 1077


In [89]:
# codigo para definir loja fixa

currentStores = ['10H1', '10I9', '10J3', '1077', '10I1', '10I4', '10J9', '10G2', '1086', '10H5', '10G6', '10J4', '10J7', '10i6', '10I8', '0091', '10J0', '10J5', '10G8', '10I3', '10J1', '10J8', '10L9', '10L7', '10L8', '10L0', '10I5', '10J2', '10M1', '10M0', '10F9', '10C4', '10M2', '1010', '1012', '1017', '1018', '1021', '1022', '1025', '1027', '1028', '1036', '1043', '1044', '1045', '1049', '1052', '1054', '1055', '1065', '1068', '1071', '1089', '10C2', '10E2', '10E4', '10E5', '10F3', '10F8']

lojaAtual = '10I1'
print('loja', lojaAtual)

loja 10I1


## Definindo data do relatório

In [3]:
# codigo para definir data aleatória
###     -> self date
import datetime
import random

maskStore = df_vendas['distributorId'] == lojaAtual
data_inicial = df_vendas[maskStore]['invoice_date'].min()
data_final = df_vendas[maskStore]['invoice_date'].max()

dias_diferenca = (data_final - data_inicial).days
dias_aleatorio = random.randint(0, dias_diferenca)
yesterday_date = data_inicial + datetime.timedelta(days=dias_aleatorio)

print('dia escolhido ', yesterday_date)
print('loja', lojaAtual)

dia escolhido  2024-10-30
loja 1077


In [91]:
# codigo para definir data fixa
yesterday_date = datetime.datetime(2024,5,15)
print('dia escolhido ', yesterday_date)
print('loja', lojaAtual)

dia escolhido  2024-05-15 00:00:00
loja 10I1


## Definir emails que estarão no relatório

In [4]:
emailRecipients = [
    'joao.garcia@ammovarejo.com.br',
    # 'rodrigo.chiesa@coteminas.com.br',
    # 'inovacao@ammovarejo.com.br'
]
print("Lista de recipients", emailRecipients)

Lista de recipients ['joao.garcia@ammovarejo.com.br']


## Rodando código para criação do relatório


In [83]:
###     -> _abreviate_vendedor_name
def _abreviate_vendedor_name(full_name: str) -> str:
        full_name = full_name.replace("   ", " ")
        full_name = full_name.replace("  ", " ")

        if len(full_name.title().split(" ")) > 1:
            splited_name = full_name.title().split(" ")
            nome_completo = splited_name[0]
            splited_name.pop(0)

            for name in splited_name:
                nome_completo += " " + name[0] + "."

            return nome_completo

        return full_name

###     -> _build_df_vendas_nf_pdv
df_vendas_temp = df_vendas.copy()

PDV_TYPE_LIST = ["STORE", "PI - 360", "PI", "EP", "REFUND"]

mask_pdv = df_vendas_temp["tipo_transacao"].isin(PDV_TYPE_LIST)

df_vendas_pdv1 = df_vendas_temp[mask_pdv]
df_vendas_pdv1.loc[:, "tipo_transacao"] = df_vendas_pdv1["tipo_transacao"].replace("PI - 360", "STORE")
df_vendas_pdv1.loc[:, "tipo_transacao"] = df_vendas_pdv1["tipo_transacao"].replace("PI", "STORE")
df_vendas_pdv1.loc[:, "tipo_transacao"] = df_vendas_pdv1["tipo_transacao"].replace("EP", "STORE")

df_vendas_pdv1 = df_vendas_pdv1.reset_index(drop=True)

###     -> _build_df_parcelas_with_displaycode(self):
df_vendedor_displaycode = (
    df_vendas_pdv1.groupby(["displaycode", "cpf_vendedor_inteiro", "distributorId"])["year"]
        .count()
        .reset_index()
        .drop(columns="year")
)

df_parcelas_with_displaycode = df_parcelas.merge(df_vendedor_displaycode, how="left", on="displaycode")

###     -> atualiza o df_nome_vendedor com os dados de nome e cpf da tabela df_vendedores
df_nome_vendedor = df_vendedores[["name", "cpf"]]
df_nome_vendedor.columns = ["vendedor", "cpf_vendedor_inteiro"]
df_nome_vendedor.loc[:, "vendedor"] = df_nome_vendedor["vendedor"].apply(_abreviate_vendedor_name)


###     -> _build_masks
def build_masks(kpi_type, kpi_period):
    mask_tickets = pd.Series(np.ones(df_vendas_pdv1.shape[0]).astype(bool))
    mask_vendas = pd.Series(np.ones(df_vendas_pdv1.shape[0]).astype(bool))
    mask_parcelas = pd.Series(np.ones(df_parcelas_with_displaycode.shape[0]).astype(bool))

    if kpi_type == "loja":
        mask_meta = df_metas["tipo_meta"] == "Meta Loja PDV"
    elif kpi_type == "vendedor":
        mask_meta = df_metas["tipo_meta"] == "Meta Vendedor PDV"

    if kpi_period == "Dia":
        mask_yesterday_vendas = df_vendas_pdv1["invoice_date"] == yesterday_date
        mask_vendas = mask_yesterday_vendas
        mask_tickets = mask_tickets & mask_yesterday_vendas

        mask_metas_yesterday = df_metas["date"] == yesterday_date
        mask_meta = mask_meta & mask_metas_yesterday

        mask_parcelas = df_parcelas_with_displaycode["data_emissao_NF"] == yesterday_date

    return mask_tickets, mask_vendas, mask_parcelas, mask_meta

###     -> _build_kpis_dataframe -> _merge_dataframes
def _build_kpis_dataframe(kpi_type, kpi_period, mask_vendas,mask_tickets,mask_parcelas,mask_meta):
    AGG_DICT = {"discount_value": "sum", "net_value": "sum", "store_cost": "sum"}

    GROUP_BY_COLUMNS_DICT = {
        "Ano": ["distributorId", "year"],
        "Mês": ["distributorId", "year", "month"],
        "Dia": ["distributorId", "year", "month"],
    }

    group_by_columns = GROUP_BY_COLUMNS_DICT[kpi_period]
    if kpi_type == "vendedor":
        group_by_columns = ["cpf_vendedor_inteiro"] + group_by_columns

    df_kpis: pd.DataFrame = df_vendas_pdv1[mask_vendas].groupby(group_by_columns).agg(AGG_DICT).reset_index()

    df_kpis = df_kpis.merge(
        df_parcelas_with_displaycode[mask_parcelas].groupby(group_by_columns)["n_parcelas"].mean().reset_index(),
            on=group_by_columns,
            how="left",
        )

    df_kpis = df_kpis.merge(
        df_vendas_pdv1[mask_tickets].groupby(group_by_columns)["parent_orderId"].nunique().reset_index().rename(columns={"parent_orderId": "tkts"}),
        how="left",
        )

    df_kpis = df_kpis.merge(
        df_vendas_pdv1[mask_tickets].groupby(group_by_columns)["amount"].sum().reset_index(), 
        how="left"
    )

    df_kpis = df_kpis.merge(
        df_metas[mask_meta].groupby(group_by_columns)["meta"].sum().reset_index(), 
        how="left"
    )

    if kpi_type == "vendedor":
        df_kpis = df_kpis.merge(
        df_vendas_pdv1.groupby(group_by_columns)["invoice_date"]
        .nunique()
        .reset_index()
        .rename(columns={"invoice_date": "days_of_work"}),
        how="left",
        )

    if kpi_period == "Dia":
        df_kpis["Dia"] = yesterday_date.day

    if kpi_type == "loja":
        if kpi_period == "Ano":
            df_kpis["tkts_per_day"] = df_kpis["tkts"] / yesterday_date.timetuple().tm_yday
        if kpi_period == "Mês":
            df_kpis["tkts_per_day"] = df_kpis["tkts"] / yesterday_date.day
        if kpi_period == "Dia":
            df_kpis["tkts_per_day"] = df_kpis["tkts"]

    elif kpi_type == "vendedor":
        if kpi_period == "Mês":
            df_kpis["tkts_per_day"] = df_kpis["tkts"] / df_kpis["days_of_work"].round(2)
        else:
            df_kpis["tkts_per_day"] = df_kpis["tkts"]

    df_kpis["type"] = kpi_period

    df_kpis["mkp"] = (df_kpis["net_value"] / df_kpis["store_cost"]).round(2)
    df_kpis["tkt_medio"] = df_kpis["net_value"] / df_kpis["tkts"].round(2)
    df_kpis["meta_percentual"] = (df_kpis["net_value"] / df_kpis["meta"] * 100).round(2)
    df_kpis["PA"] = df_kpis["amount"] / df_kpis["tkts"].round(2)
    df_kpis["discount_percentual"] = ((df_kpis["discount_value"] / df_kpis["net_value"]) * 100).round(2)
    
    return df_kpis


###     -> cria um dicionário de dataframes kpis para cada período para as lojas
###     -> cria os dataframes de kpi para vendedor e para lojas

kpis_loja_dict = {}
for kpi_period in ["Dia", "Mês", "Ano"]:
    mask_vendas,mask_tickets,mask_parcelas,mask_meta = build_masks(kpi_period=kpi_period, kpi_type='loja')
    kpis_loja_dict[f"kpis_loja_{kpi_period.lower()}"] = _build_kpis_dataframe(
                kpi_type="loja", 
                kpi_period=kpi_period,
                mask_vendas=mask_vendas,
                mask_tickets=mask_tickets,
                mask_parcelas=mask_parcelas,
                mask_meta=mask_meta
            )


kpis_vendedor_dict = {}
for kpi_period in ["Dia", "Mês", "Ano"]:
    mask_vendas,mask_tickets,mask_parcelas,mask_meta = build_masks(kpi_period=kpi_period, kpi_type='vendedor')
    kpis_vendedor_dict[f"kpis_vendedor_{kpi_period.lower()}"] = _build_kpis_dataframe(
        kpi_type="vendedor", 
        kpi_period=kpi_period,
        mask_vendas=mask_vendas,
        mask_tickets=mask_tickets,
        mask_parcelas=mask_parcelas,
        mask_meta=mask_meta
    )

df_kpis_loja = pd.concat(list(kpis_loja_dict.values()))
df_kpis_vendedor = pd.concat(list(kpis_vendedor_dict.values()))
df_kpis_vendedor = df_kpis_vendedor.dropna(subset="cpf_vendedor_inteiro")
mask = df_kpis_vendedor["cpf_vendedor_inteiro"] != "Devolução"
df_kpis_vendedor = df_kpis_vendedor[mask]

###     -> format all tables
#######    -> format store kpis
def format_store_kpis(id_loja):
    mask = df_kpis_loja["distributorId"] == id_loja

    df_store = df_kpis_loja[mask].fillna(0).copy()
    df_store_current_month = df_store.loc[df_store.groupby(["type"])["month"].idxmax()].drop_duplicates()

    rename_dict = {
        "net_value": "Venda Loja (PDV)",
        "meta": "Meta Loja (PDV)",
        "meta_percentual": "Meta % (PDV)",
        "mkp": "Markup",
        "tkts_per_day": "Tickets/Dia",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        # 'n_parcelas':'Parcela Média',
        "discount_percentual": "Desconto Médio %",
        "type": "Período",
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Venda Loja (PDV)": format_money,
        "Meta Loja (PDV)": format_money,
        "Meta % (PDV)": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets/Dia": "{:,.1f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        # 'Parcela Média':'{:,.1f}',
        "Desconto Médio %": format_percentage,
    }

    df_store_formated = df_store_current_month[rename_dict.keys()].round(2).rename(columns=rename_dict)

    for key, format in format_dict.items():
        df_store_formated[key] = df_store_formated[key].map(format.format)

        if format == format_money:
            df_store_formated[key] = df_store_formated[key].astype(str).str.replace(",", ".")
    try:
        df_store_formated = df_store_formated.set_index("Período").reset_index().T[[1, 2, 0]]
    except KeyError:
        df_store_formated = pd.DataFrame()
    
    return df_store_formated

###     -> format all tables
#######    -> format_vendedor_kpis_mes
def format_vendedor_kpis_mes(id_loja):
    mask = df_kpis_vendedor["distributorId"] == id_loja
    df_vendedor = df_kpis_vendedor[mask].reset_index(drop=True)
    df_vendedor["month"] = df_vendedor["month"].fillna(0)

    mask_type = df_vendedor["type"].isin(["Mês"])
    mask_mes = df_vendedor["month"] == yesterday_date.month
    mask_ano = df_vendedor["year"] == yesterday_date.year
    mask_completa = mask_type & mask_mes & mask_ano
    df_vendedor_mes: pd.DataFrame = df_vendedor[mask_completa].reset_index(drop=True)
    df_vendedor_mes["normalized_net_value"] = (
        df_vendedor_mes["net_value"] * 100 / round(df_vendedor_mes["net_value"].max(), 2)
    )
    df_vendedor_mes["net_value_share"] = (
        df_vendedor_mes["net_value"] * 100 / round(df_vendedor_mes["net_value"].max(), 2)
    )
    df_vendedor_mes = df_nome_vendedor.merge(df_vendedor_mes, on="cpf_vendedor_inteiro").drop(
        columns="cpf_vendedor_inteiro"
    )

    rename_dict = {
        "vendedor": "Vendedor",
        "days_of_work": "Dias Trabalhados",
        "net_value": "Acumulado no Mês",
        "normalized_net_value": "Homogeneidade",
        "meta": "Meta do Mês",
        "meta_percentual": "Meta %",
        "mkp": "Markup",
        "tkts_per_day": "Tickets/Dia",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        # 'n_parcelas':'Parcela Média',
        "discount_percentual": "Desconto Médio %",
        # 'type':'Período'
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Acumulado no Mês": format_money,
        "Homogeneidade": format_percentage,
        "Meta do Mês": format_money,
        "Meta %": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets/Dia": "{:,.1f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        "Desconto Médio %": format_percentage,
    }

    df_vendedor_mes_formatado = (
        df_vendedor_mes[rename_dict.keys()]
        .sort_values("normalized_net_value", ascending=False)
        .round(2)
        .rename(columns=rename_dict)
    )

    for key, format in format_dict.items():
        df_vendedor_mes_formatado[key] = df_vendedor_mes_formatado[key].map(format.format)
        if format == format_money:
            df_vendedor_mes_formatado[key] = df_vendedor_mes_formatado[key].astype(str).str.replace(",", ".")

    df_vendedor_mes_formatado = df_vendedor_mes_formatado.reset_index(drop=True).T.reset_index().T

    return df_vendedor_mes_formatado.set_index(0)

###     -> format all tables
#######    -> format_vendedor_kpis_dia
def format_vendedor_kpis_dia(id_loja):
    mask = df_kpis_vendedor["distributorId"] == id_loja
    df_vendedor = df_kpis_vendedor[mask].reset_index(drop=True)
    df_vendedor["month"] = df_vendedor["month"].fillna(0)

    mask_type = df_vendedor["type"].isin(["Dia"])
    mask_mes = df_vendedor["month"] == yesterday_date.month
    mask_ano = df_vendedor["year"] == yesterday_date.year
    mask_dia = df_vendedor['Dia'] == yesterday_date.day
    df_vendedor_dia = df_vendedor[mask_dia & mask_mes & mask_ano & mask_type].reset_index(drop=True)
    
    df_vendedor_dia["normalized_net_value"] = (
        df_vendedor_dia["net_value"] * 100 / round(df_vendedor_dia["net_value"].max(), 2)
    )
    df_vendedor_dia["net_value_share"] = (
        df_vendedor_dia["net_value"] * 100 / round(df_vendedor_dia["net_value"].max(), 2)
    )
    df_vendedor_dia = df_nome_vendedor.merge(df_vendedor_dia, on="cpf_vendedor_inteiro").drop(
        columns="cpf_vendedor_inteiro"
    )

    rename_dict = {
        "vendedor": "Vendedor",
        "net_value": "Acumulado no Dia",
        "normalized_net_value": "Homogeneidade",
        "meta": "Meta do Dia",
        "meta_percentual": "Meta %",
        "mkp": "Markup",
        "tkts_per_day": "Tickets",
        "PA": "Peças por Atendimento",
        "tkt_medio": "Ticket Médio",
        # 'n_parcelas':'Parcela Média',
        "discount_percentual": "Desconto Médio %",
    }

    format_money = "R$ {:,.0f}"
    format_percentage = "{:,.0f}%"

    format_dict = {
        "Acumulado no Dia": format_money,
        "Homogeneidade": format_percentage,
        "Meta do Dia": format_money,
        "Meta %": format_percentage,
        "Markup": "{:,.2f}",
        "Tickets": "{:,.0f}",
        "Peças por Atendimento": "{:,.1f}",
        "Ticket Médio": format_money,
        # 'Parcela Média':'{:,.1f}',
        "Desconto Médio %": format_percentage,
    }

    df_vendedor_dia_formatado = (
        df_vendedor_dia[rename_dict.keys()]
        .sort_values("normalized_net_value", ascending=False)
        .rename(columns=rename_dict)
    )

    for key, format in format_dict.items():
        df_vendedor_dia_formatado[key] = df_vendedor_dia_formatado[key].map(format.format)
        if format == format_money:
            df_vendedor_dia_formatado[key] = df_vendedor_dia_formatado[key].astype(str).str.replace(",", ".")

    df_vendedor_dia_formatado = df_vendedor_dia_formatado.reset_index(drop=True).T.reset_index().T
    return df_vendedor_dia_formatado.set_index(0)

###     -> format all tables
#######    -> format_all_tables
def format_all_tables(id):
    df_store_formated = format_store_kpis(id)
    
    if df_store_formated.size == 0:
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

    df_vendedor_formated_month = format_vendedor_kpis_mes(id)

    df_vendedor_formated_day = format_vendedor_kpis_dia(id)

    return df_store_formated.T.set_index('Período').T, df_vendedor_formated_month.T.reset_index(drop=True).set_index("Vendedor").T, df_vendedor_formated_day.T.reset_index(drop=True).set_index("Vendedor").T


loja, mes, dia = format_all_tables(lojaAtual)
print('Loja: ', lojaAtual)
print('Data: ', yesterday_date)

Loja:  1077
Data:  2024-10-30


## Criando html

In [84]:
relatorioDiaStr = yesterday_date.strftime("%d-%m-%Y")

emailBody = ""

htmlValue = """
<html>
  <head></head>
  <body>
    <h1>Relatorio loja - {1}</h1>
    <h2>Data: {2}</h2>
    {0}
  </body>
  <br/><br/>
</html>
""".format(loja.to_html(), lojaAtual, yesterday_date.strftime('%d-%m-%Y'))
emailBody += htmlValue

htmlValue = """
<html>
  <head></head>
  <body>
  <h1>Relatorio vendedor - Visão diária</h1>
  <p>Loja:{1} | Data: {2}</p>
    {0}
  </body>
  <br/><br/>
</html>
""".format(dia.to_html(),lojaAtual, yesterday_date.strftime('%d-%m-%Y'))
emailBody += htmlValue

htmlValue = """
<html>
  <head></head>
  <body>
  <h1>Relatorio vendedor - Visão mensal</h1>
  <p>Loja:{1} | Data: {2}</p>
    {0}
  </body>
  <br/><br/>
</html>
""".format(mes.to_html(),lojaAtual, yesterday_date.strftime('%m-%Y'))
emailBody += htmlValue
emailBody

'\n<html>\n  <head></head>\n  <body>\n    <h1>Relatorio loja - 1077</h1>\n    <h2>Data: 30-10-2024</h2>\n    <table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Período</th>\n      <th>Dia</th>\n      <th>Mês</th>\n      <th>Ano</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Venda Loja (PDV)</th>\n      <td>R$ 3.528</td>\n      <td>R$ 120.262</td>\n      <td>R$ 856.481</td>\n    </tr>\n    <tr>\n      <th>Meta Loja (PDV)</th>\n      <td>R$ 2.044</td>\n      <td>R$ 132.119</td>\n      <td>R$ 1.355.626</td>\n    </tr>\n    <tr>\n      <th>Meta % (PDV)</th>\n      <td>173%</td>\n      <td>91%</td>\n      <td>63%</td>\n    </tr>\n    <tr>\n      <th>Markup</th>\n      <td>5.69</td>\n      <td>4.89</td>\n      <td>4.30</td>\n    </tr>\n    <tr>\n      <th>Tickets/Dia</th>\n      <td>2.0</td>\n      <td>4.1</td>\n      <td>3.5</td>\n    </tr>\n    <tr>\n      <th>Peças por Atendimento</th>\n      <td>2.5</td>\n      <td>2.8</td>\n     

## Enviando email

In [85]:
import os
import smtplib
from dataclasses import dataclass
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from typing import Any, Callable, Dict, List, Optional, Union
from commom.logger import logger
from commom.base_classes.base_sender import BaseMessenger
from innovation_messenger.config import config

@dataclass
class IEmailProperties:
    subject: str
    body: str
    recipient: Union[str, List[str]]
    file_name: Optional[str] = None


def send_email_html(email_properties: IEmailProperties) -> bool:
    message = MIMEMultipart()
    message["Subject"] = email_properties.subject
    message["From"] = 'inovacao@ammovarejo.com.br'
    messageTo: str

    if isinstance(email_properties.recipient, List):
        messageTo = ", ".join(email_properties.recipient)
    else:
        messageTo = email_properties.recipient

    message["To"] = messageTo
    html_part = MIMEText(email_properties.body, 'html')
    message.attach(html_part)
    if isinstance(email_properties.recipient, List):
        messageTo = ", ".join(email_properties.recipient)
    else:
        messageTo = email_properties.recipient
        
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        try:
            server.login(user='inovacao@ammovarejo.com.br', password='inovaAM!05')
            server.sendmail('inovacao@ammovarejo.com.br', email_properties.recipient, message.as_string())
            return True
        except Exception as e:
            logger.error(e)
            return False


email_properties = IEmailProperties(
    subject=f'Teste - Base antiga - Loja {lojaAtual} - Dia {relatorioDiaStr}',
    recipient=emailRecipients,
    body=emailBody,
)

send_email_html(
  email_properties=email_properties,
)


True